In [17]:
import re
import numpy as np
import pandas as pd
from pprint import pprint
from nltk.tokenize import regexp_tokenize, wordpunct_tokenize
from nltk.tokenize import RegexpTokenizer 
from nltk.stem import WordNetLemmatizer
import string
import nltk
nltk.download('stopwords')
import os
import fnmatch
import io
from pprint import pprint
from time import time  # to time our operations

#LDA
import warnings
warnings.filterwarnings('ignore')
import os 
%matplotlib inline
import pyLDAvis.gensim
import gensim
pyLDAvis.enable_notebook()


# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess, tokenize
from gensim.models import CoherenceModel

def stopwords_city(City):
    month = ['january', 'february', 'march', 'april', 'may', 'june', 'july', 'august', 'september', 'october', 'november', 'december' ]
    day = ['monday', 'tuesday', 'wednesday', 'thursday', 'friday', 'saturday', 'sunday']
    if City == 'Boston':
        return month + day + ['boston', 'roxbury','fiandaca','opencounter', 'accela','forth', 'ortega', 'burro','theroxburyinnovation', 'julie', 'elliott', '21stcentury', 'nstar','dorsey','austin', 'blackmon', 'swett', 'ofboston', 'paired','ass','first', 'martin', 'addition', 'every', 'walsh', 'according', 'please visit', 'noted',
            'please','april', 'february', 'visit', 'across', 'december', 'https', 'following',
             'noted', 'along', 'three', 'total', 'pointed', 'citys', 'announced', 'massachusetts', 'million']
    if City =='Taipei':
        return month + day +['taipei','since', 'dadaocheng', 'serf', 'addition', 'february','related', 'october', 'january', 'around', 'taking', 'march', 'first', 'according', 'please visit',
                'noted', 'please','april', 'among''february', 'visit', 'across', 'december', 'https', 'ko',
                'following', 'universiade','september','november','august','noted', 'along', 'three', 'total', 'pointed', 'citys', 'chinese']
    if City == 'Seoul':
        return month + day +['seoul','soon','yeomchang', 'dulle','three', 'gaehwasinnonhyeon', 'major', 'first', 'million', 'various','selected','around','number', 'based','total',
                'order','special', 'since']
    if City == 'Helsinki':
        return month + day + ['helsinki','based', '6aika', 'wrtsil','january', 'first', 'sitra']
    else:
        return month + day
    
def corpus(City):

    folder = '/Users/clementnicolas/JournalPaper3/{}_News'.format(City)

    corpus=[]
    tokenizer = RegexpTokenizer(r'\w+')
    stopwords = nltk.corpus.stopwords.words('english')
    stopwords.extend(stopwords_city(City))
    with_stopwords=0
    without_stopwords=0
    articles=0
    tokens =0
    for file in os.listdir(folder):
        
        path = os.path.join(folder, file)
        if fnmatch.fnmatch(file, '*.txt'):
            with io.open(path, 'r', encoding="UTF-8") as fin:
                content = fin.read()
                data = content.splitlines()
                #articles=articles+len(data)


            for sentence in data:
                word_list = tokenizer.tokenize(sentence.lower())
                word_list1 = [word for word in word_list if word.isalpha()]
                word_list2 = [word for word in word_list1 if len(word)>3]
                word_list3 = [word for word in word_list2 if word not in stopwords]
                if len(word_list3)>20:
                    corpus.append(word_list3)
                    #with_stopwords=with_stopwords+len(word_list2)
                    #without_stopwords = without_stopwords+len(word_list3)
                    #tokens = tokens+len(word_list)
    
    
    # Build the bigram and trigram models
    bigram = gensim.models.Phrases(corpus, min_count=5, threshold=100)  # higher threshold fewer phrases.
    trigram = gensim.models.Phrases(bigram[corpus], threshold=100)

    # Faster way to get a sentence clubbed as a trigram/bigram
    bigram_mod = gensim.models.phrases.Phraser(bigram)
    trigram_mod = gensim.models.phrases.Phraser(trigram)

    def make_bigrams(texts):
        return [bigram_mod[doc] for doc in texts]

    def make_trigrams(texts):
        return [trigram_mod[bigram_mod[doc]] for doc in texts]

    ######## For BIGRAMS
    corpus2 = make_bigrams(corpus)
    
    ######## For TRIGRAMS
    
    #corpus22 = make_bigrams(corpus)
    #corpus2 = make_trigrams(corpus22)

    #print(corpus2[:10])
    L=[]
    lemmatizer = WordNetLemmatizer()
    for w in range(len(corpus2)):
        x = 0
        while x <(len(corpus2[w])):
            #if corpus2[w][x] not in L:
            #    L.append(corpus2[w][x])
            corpus2[w][x] = lemmatizer.lemmatize(corpus2[w][x])
            if corpus2[w][x] in stopwords:
                del corpus2[w][x]
            x += 1
                
    #print(len(L))
    #print(articles)
    #print(with_stopwords)
    #print(without_stopwords)
    #print(tokens)
    return corpus2

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/clementnicolas/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [18]:
import pandas as pd
from gensim import models,corpora
import pyLDAvis.gensim
from gensim.models.coherencemodel import CoherenceModel
import warnings

In [19]:
pd.set_option('max_colwidth',400)
pyLDAvis.enable_notebook()
warnings.filterwarnings('ignore')

In [51]:
City = "Taipei"
NumTopics=11

Lda = gensim.models.LdaModel
data_lemmatized = corpus(City)
lda_final =Lda.load('lda_final.{}'.format(City))
dictionary = corpora.Dictionary.load('dictionary.{}'.format(City))
doc_term_matrix = corpora.MmCorpus('doc_term_matrix.{}.mm'.format(City))

#dictionary = corpora.Dictionary(corpus(City))
#doc_term_matrix = [dictionary.doc2bow(doc) for doc in corpus(City)]

In [52]:
a = lda_final.show_topics(num_topics=NumTopics,formatted=False,num_words=10)
#b = lda_final.top_topics(doc_term_matrix,texts=None, dictionary=dictionary,topn=10) # This orders the topics in the decreasing order of coherence score

b = lda_final.top_topics(doc_term_matrix,texts=data_lemmatized, dictionary=dictionary,coherence='c_v', topn=10) 


print(b)
topic2skillb = {}
topic2csb = {}
topic2skilla = {}
topic2csa = {}
num_topics =lda_final.num_topics
cnt =1

for ws in b:
    wset = set(w[1] for w in ws[0])
    topic2skillb[cnt] = wset
    topic2csb[cnt] = ws[1]
    cnt +=1
print(topic2csb)
for ws in a:
    wset = set(w[0]for w in ws[1])
    topic2skilla[ws[0]+1] = wset
    
for i in range(1,num_topics+1):
    for j in range(1,num_topics+1):  
        if topic2skilla[i].intersection(topic2skillb[j])==topic2skilla[i]:
            topic2csa[i] = topic2csb[j]

finalData = pd.DataFrame([],columns=['Topic','words'])
finalData['Topic']=topic2skilla.keys()
finalData['Topic'] = finalData['Topic'].apply(lambda x: 'Topic'+str(x))
finalData['words']=topic2skilla.values()
finalData['cs'] = topic2csa.values()
finalData.sort_values(by='cs',ascending=False,inplace=True)
finalData.to_csv('CoherenceScore.csv')
finalData

[([(0.0467507, 'health'), (0.040605254, 'care'), (0.039206658, 'hospital'), (0.027241489, 'medical'), (0.026323328, 'medicine'), (0.025486015, 'doctor'), (0.02059296, 'staying'), (0.01732103, 'center'), (0.012934786, 'home'), (0.01245359, 'senior')], 0.6769067232140114), ([(0.063640326, 'easycard'), (0.043704428, 'station'), (0.031522293, 'metro'), (0.02241513, 'hotline'), (0.02050583, 'service'), (0.019232634, 'trtc'), (0.019018421, 'corporation'), (0.01826394, 'line'), (0.01818784, 'arena'), (0.01700705, 'call')], 0.6408218415300354), ([(0.040061004, 'project'), (0.023925278, 'construction'), (0.020677662, 'building'), (0.019116169, 'public'), (0.017733173, 'plan'), (0.014960574, 'resident'), (0.01420591, 'housing'), (0.013739538, 'government'), (0.013607428, 'facility'), (0.012844843, 'work')], 0.5848580814557329), ([(0.026278157, 'road'), (0.02141704, 'station'), (0.013497829, 'area'), (0.011723779, 'park'), (0.011620505, 'parking'), (0.009609567, 'water'), (0.009418641, 'traffic')

,Topic,words,cs
6,Topic7,"{health, home, senior, hospital, medicine, staying, care, medical, doctor, center}",0.676907
4,Topic5,"{metro, corporation, line, trtc, hotline, call, arena, station, easycard, service}",0.640822
10,Topic11,"{facility, housing, public, project, work, plan, building, government, resident, construction}",0.584858
5,Topic6,"{road, park, area, take, line, traffic, route, station, water, parking}",0.582821
0,Topic1,"{citizen, public, department, system, policy, agency, city, service, government, information}",0.525019
9,Topic10,"{education, people, forum, mayor, year, city, school, government, issue, student}",0.468027
1,Topic2,"{international, department, member, event, city, mayor, year, government, also, taiwan}",0.467037
8,Topic9,"{free, event, tuna, food, enjoy, district, exhibition, also, information, visitor}",0.390772
7,Topic8,"{runner, business, platform, company, tamsui_river, category, wide_range, service, singapore, sign_language}",0.387203
3,Topic4,"{article, book, class, family, child, monthly, definitely, registration, tpedoits, animal}",0.373572


In [53]:
num_topics =NumTopics
vis = pyLDAvis.gensim.prepare(lda_final, doc_term_matrix, dictionary,sort_topics=False)
pyLDAvis.save_html(vis,f'pyLDAvis_{num_topics}.html')
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.134617  0.103809       1        1  12.776336
1     -0.223022 -0.086878       2        1  26.051527
2      0.041827 -0.183458       3        1   5.506674
3      0.132461  0.026438       4        1   3.985045
4      0.217565  0.027294       5        1   2.926272
5      0.010142 -0.086872       6        1  10.788286
6      0.168397  0.129897       7        1   2.230400
7      0.121280  0.156298       8        1   3.015632
8      0.042555 -0.288374       9        1  11.606492
9     -0.239229  0.069892      10        1  14.976151
10    -0.137359  0.131954      11        1   6.137191, topic_info=             Term         Freq        Total Category  logprob  loglift
20           city  6049.000000  6049.000000  Default  30.0000  30.0000
86          mayor  3128.000000  3128.000000  Default  29.0000  29.0000
488       station  1191.000000  1191.000000  Default  28.0000  28.0000
121       service  1526.000000  1526.000000  Default  27.0000  27.0000
1185         park  1091.000000  1091.000000  Default  26.0000  26.0000
...           ...          ...          ...      ...      ...      ...
2207  implemented    94.827187   209.548462  Topic11  -5.2485   1.9979
47         future    98.853432   359.083954  Topic11  -5.2069   1.5009
2850        green    95.366676   226.115265  Topic11  -5.2428   1.9275
157          area   103.774345   893.771057  Topic11  -5.1584   0.6376
86          mayor   104.488632  3128.470703  Topic11  -5.1515  -0.6084

[634 rows x 6 columns], token_table=       Topic      Freq            Term
term                                  
4801      10  0.991741           abuse
244        2  0.205366        activity
244        3  0.053574        activity
244        4  0.196437        activity
244        8  0.002232        activity
...      ...       ...             ...
5276       6  0.279084         youbike
10942      5  0.968176  youbike_rental
1852       2  0.994920           youth
4341       6  0.983114       zhongxiao
5405       4  0.984646             zoo

[1069 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11])

In [54]:
token_percent = vis.topic_coordinates.sort_values(by='topics').loc[:,['topics','Freq']]
df = token_percent.iloc[:,[1]]
df["Topic"]=1
for i in range(len(df)):
    df["Topic"][i]="Topic"+str(i+1)
df

,Freq,Topic
topic,,
0,12.776336,Topic1
1,26.051527,Topic2
2,5.506674,Topic3
3,3.985045,Topic4
4,2.926272,Topic5
5,10.788286,Topic6
6,2.230400,Topic7
7,3.015632,Topic8
8,11.606492,Topic9


In [55]:
def get_relevant_words(vis,lam=0.3,topn=10):
    a = vis.topic_info
    a['finalscore'] = a['logprob']*lam+(1-lam)*a['loglift']
    a = a.loc[:,['Category','Term','finalscore']].groupby(['Category'])\
    .apply(lambda x: x.sort_values(by='finalscore',ascending=False).head(topn))
    a = a.loc[:,'Term'].reset_index().loc[:,['Category','Term']]
    a = a[a['Category']!='Default']
    a = a.to_dict('split')['data']
    d ={}
    for k,v in a: 
        if k not in d.keys():
            d[k] =set()
            d[k].add(v)
        else:
            d[k].add(v)
    finalData = pd.DataFrame([],columns=['Topic','words with Relevance'])
    finalData['Topic']=d.keys()
    finalData['words with Relevance']=d.values()
    return finalData

In [56]:
#get_relevant_words(vis,0.3).merge(finalData,how='left',on ='Topic').sort_values(by='cs',ascending=False).iloc[:,[0,1]].merge(token_percent)
a = get_relevant_words(vis,0.2,20).merge(finalData,how='left',on ='Topic').merge(df, how='left', on = 'Topic').sort_values(by='Freq',ascending=False)
a.rename(columns={'cs':'Coherence','Freq':'Frequency', 'words with Relevance':'Relevant Words'}, inplace=True)
b = a[['Topic', 'Frequency','Relevant Words', 'Coherence']]
#b.to_csv('WordGrid.Seoul.csv')


In [57]:
b

,Topic,Frequency,Relevant Words,Coherence
3,Topic2,26.051527,"{competition, commissioner, participation, delegation, address, group, cultural, chen, local, international, sport, member, team, milk, japan, charity, association, organization, youth, osaka}",0.467037
1,Topic10,14.976151,"{publication, mayor, student, asked, said, forum, embracing, advice, citing, belief, research, challenge, hong_kong, sharing, education, people, training, research_development, would, shanghai}",0.468027
0,Topic1,12.776336,"{survey, household, rdec, hereinafter_referred, procedure, fee, application, owner, quality, system, user, poll, smart, satisfy, evaluation_commission, data, respondent, open_data, fried, service}",0.525019
10,Topic9,11.606492,"{restaurant, airline, ingredient, shin_kong, style, tuna, sauce, organizer, traditional, tpedoit, shabu, read, reader, flavor, spicy, delicacy, performer, blue, latest_pictorial, delicious}",0.390772
7,Topic6,10.788286,"{intersection, traffic, route, bridge, cycling, wheel, road, minzu, lane, avoid, vehicle, train, closed, staircase, driver, riverside, alley_lane, bicycle, drill, parking}",0.582821
2,Topic11,6.137191,"{integrate, kitchen, project, proposal, unit, property, railway, urban, housing, urban_renewal, analysis, building, chou, borough, construction, administrative_efficiency, shezidao, plan, slated, typhoon}",0.584858
4,Topic3,5.506674,"{beitou, flower, paradise, relaxing, cabbage, tree, park, photography, aiph, spring, cuisine, pot, wonderful, wenshan, landscape, temple, farm, flora_expo, prefer, pickled}",0.359022
5,Topic4,3.985045,"{bought, article, family, child, represented, endangered, kid, conservation, tpedoits, book, wetland, class, parent, editor, monthly, insurance, zoo, submission, definitely, registration}",0.373572
9,Topic8,3.015632,"{urgent, resume, consultation, tamsui_river, incentive, startup, category, sign_language, woman, suspend, device, wide_range, index, runner, fisu, fude, race, marathon, singapore, biotech}",0.387203
6,Topic5,2.926272,"{aged_older, rapid_transit, corporation_trtc, fare, auctioneer, percent_discount, card, pas, maokong_gondola, arena, taoyuan, bidder, trtc, hearing_impaired, discount, easycard, metro, hotline, airport, maokong}",0.640822
